In [7]:
import requests
import datetime
import json
import plotly
import plotly.graph_objs as go

LCD_ENDPOINT = "https://tequila-fcd.terra.dev/v1/txs/"
CONTRACT = "terra1qzzcc8r04ylr20lwzhlafwynqtmjdnrzlc35jh"

LCD_ENDPOINT = "https://fcd.terra.dev/v1/txs/"
CONTRACT = "terra1mtwph2juhj0rvjz7dy92gvl6xvukaxu8rfv8ts"


def fetch_txs():
    txs = []
    params = {
        "account": CONTRACT,
        "limit": 100
    }
    i = 0
    until_day = datetime.date(2021,5,18)
    day = datetime.date.today()
    while day>=until_day:
        resp = requests.get(LCD_ENDPOINT, params=params)
        txs = txs + resp.json()['txs']
        n = resp.json()['next']
        params['offset']=n
        i += 1
        day = datetime.datetime.strptime(
            resp.json()['txs'][0]["timestamp"], "%Y-%m-%dT%H:%M:%SZ").date()
        print(resp.json()['txs'][0]["timestamp"])

    valuable_txs = []
    for tx in txs:
        tx.pop('logs',None)
        tx.pop('raw_log',None)
        # if get_uluna(tx,CONTRACT) >0:
        #     valuable_txs.append(tx)
    return txs


def get_uluna(tx,contract):
    coins = [msg['value']['coins'] for msg in tx['tx']['value']['msg'] if msg['value']['contract']==contract]
    if len(coins)>0 and len(coins[0]) > 0:
        if coins[0][0]['denom'] == 'uluna':
            return int(coins[0][0]['amount'])
    return 0


def funds_per_day(tx_list):
    funds = {}
    for tx in tx_list:
        day = datetime.datetime.strptime(
            tx["timestamp"], "%Y-%m-%dT%H:%M:%SZ")
        if day not in funds:
            funds[day] = 0
        uluna = get_uluna(tx,CONTRACT)
        # print(tx["timestamp"],uluna)
        funds[day] = funds[day] + uluna
    return funds

if True:
    txs = fetch_txs()
    fd = open("txs-dump.json","w")
    fd.write(json.dumps(txs))
    fd.close()
else:
    fd = open("txs-dump.json")
    txs = json.loads(fd.read())


days = []
sums = []
for k, v in funds_per_day(txs).items():
    days.append(k)
    sums.append(v)

fig = go.Figure()
fig.add_trace(go.Scatter(x=days, y=sums, name = "Суммы по дням"))

fig.show()

2021-08-20T10:31:32Z
2021-08-20T07:06:34Z
2021-08-20T04:22:07Z
2021-08-20T01:32:59Z
2021-08-19T22:47:31Z
2021-08-19T20:07:28Z
2021-08-19T17:57:43Z
2021-08-19T16:21:41Z
2021-08-19T14:32:04Z
2021-08-19T12:58:22Z
2021-08-19T11:08:30Z
2021-08-19T09:22:14Z
2021-08-19T07:51:59Z
2021-08-19T06:25:46Z
2021-08-19T05:17:34Z
2021-08-19T04:17:56Z
2021-08-19T02:37:16Z
2021-08-19T01:31:33Z
2021-08-19T00:16:42Z
2021-08-18T22:53:23Z
2021-08-18T21:26:29Z
2021-08-18T20:22:19Z
2021-08-18T19:08:37Z
2021-08-18T18:16:46Z
2021-08-18T17:03:35Z
2021-08-18T15:50:56Z
2021-08-18T14:39:57Z
2021-08-18T13:25:34Z
2021-08-18T11:31:20Z
2021-08-18T08:33:30Z
2021-08-18T05:26:51Z
2021-08-18T02:26:24Z
2021-08-17T23:46:12Z
2021-08-17T21:16:32Z
2021-08-17T18:58:37Z
2021-08-17T16:48:16Z
2021-08-17T14:23:00Z
2021-08-17T12:02:36Z
2021-08-17T10:20:15Z
2021-08-17T08:48:38Z
2021-08-17T07:18:50Z
2021-08-17T05:37:30Z
2021-08-17T03:45:36Z
2021-08-17T02:16:57Z
2021-08-16T23:12:26Z
2021-08-16T21:11:09Z
2021-08-16T19:19:23Z
2021-08-16T17

KeyError: 'coins'